# 1. Data Import and Preprocessing

## 1.1 引入数据集，生成dataframe

In [33]:
import pandas as pd
import numpy as np

data_path ='./dataset_new.csv'
columns_name = ('date', 'time', 'sensor_id', 'value', 'activity', 'status') # status是begin或end
df = pd.read_csv(data_path, names = columns_name)

In [34]:
df.dtypes

date         object
time         object
sensor_id    object
value        object
activity     object
status       object
dtype: object

In [35]:
'''
#一些测试
df.iloc[:,:4]
df_first4Attr = df.iloc[:,:4]
df[df_first4Attr.isnull().values==True]
'''

'\n#一些测试\ndf.iloc[:,:4]\ndf_first4Attr = df.iloc[:,:4]\ndf[df_first4Attr.isnull().values==True]\n'

In [36]:
df['sensor_id'].unique()

array(['M35', 'AD1-A', 'M34', 'M32', 'M33', 'M36', 'M30', 'M29', 'M37',
       'M38', 'M39', 'M41', 'M40', 'M49', 'M46', 'M50', 'M44', 'M47',
       'M48', 'M43', 'M28', 'M42', 'M27', 'M26', 'M25', 'M24', 'M21',
       'M19', 'M18', 'M17', 'M16', 'M15', 'D09', 'M14', 'M13', 'M08',
       'M07', 'M02', 'M45', 'M51', 'D08', 'AD1-B', 'AD1-C', 'D10', 'I03',
       'M06', 'M09', 'M10', 'M01', 'D12', 'L04', 'M23', 'M31', 'D15',
       'D07', 'D14', 'M11', 'M05', 'M03', 'M04', 'M22', 'M12', 'M20',
       'L11', 'D05', 'D03', 'L10', 'L06', 'L13', 'L12', 'L09'],
      dtype=object)

## 1.2 判断缺失值

In [37]:
df.dropna(axis=0, how='all', inplace=True) ## 删去了两行空白行
df

,date,time,sensor_id,value,activity,status
0,02/02/2009,7:15:17,M35,ON,R1_Bed_to_Toilet,begin
1,02/02/2009,7:15:21,AD1-A,2.82231,NaN,NaN
2,02/02/2009,7:15:23,M35,OFF,NaN,NaN
3,02/02/2009,7:15:23,M35,ON,NaN,NaN
4,02/02/2009,7:15:28,AD1-A,2.79926,NaN,NaN
...,...,...,...,...,...,...
138625,04/04/2009,7:26:16,M48,OFF,NaN,NaN
138626,04/04/2009,7:27:47,M48,ON,NaN,NaN
138627,04/04/2009,7:27:50,M47,ON,NaN,NaN
138628,04/04/2009,7:27:59,M47,OFF,NaN,NaN


## 1.3 合并date和time

In [38]:
# df.drop(df.tail(1).index, inplace = True) #从尾部去掉 1 行，这一行是异常行

In [39]:
# df["datetime"] = pd.to_datetime(df["date"] + " "+ df["time"], format='%d/%m/%Y %H:%M:%S')
df["datetime"] = pd.to_datetime(df["date"] + " "+ df["time"], dayfirst=True)

# df["datetime"]


In [40]:
# df = df.set_index("datetime") #千万不能设datetime为index! 因为datetime有重复值！！！妈的，debug半个多小时。
df

,date,time,sensor_id,value,activity,status,datetime
0,02/02/2009,7:15:17,M35,ON,R1_Bed_to_Toilet,begin,2009-02-02 07:15:17
1,02/02/2009,7:15:21,AD1-A,2.82231,NaN,NaN,2009-02-02 07:15:21
2,02/02/2009,7:15:23,M35,OFF,NaN,NaN,2009-02-02 07:15:23
3,02/02/2009,7:15:23,M35,ON,NaN,NaN,2009-02-02 07:15:23
4,02/02/2009,7:15:28,AD1-A,2.79926,NaN,NaN,2009-02-02 07:15:28
...,...,...,...,...,...,...,...
138625,04/04/2009,7:26:16,M48,OFF,NaN,NaN,2009-04-04 07:26:16
138626,04/04/2009,7:27:47,M48,ON,NaN,NaN,2009-04-04 07:27:47
138627,04/04/2009,7:27:50,M47,ON,NaN,NaN,2009-04-04 07:27:50
138628,04/04/2009,7:27:59,M47,OFF,NaN,NaN,2009-04-04 07:27:59


In [41]:
# 检查时间都是前一个<=后一个
for a, b in zip(df.datetime, df.datetime[1:]):
    if(a>b):
        print(a)

# all(a <= b for a, b in zip(df.index, df.index[1:]))

## 1.4 给begin-end包围的activity打标签，begin-end之外的标为"other"，嵌套的按照最近的begin-end来标，标完之后删掉begin、end这一列。

In [42]:
# 将"value"列的"ON"和"OPEN"转为1，"OFF"和"CLOSE"转为0

# df["value"].unique() #检查value都有哪些名称

one_vals = ["ON","OPEN", "PRESENT"]
zero_vals = ["OFF","CLOSE", "ABSENT"]

df.loc[df['value'].isin(one_vals), 'value'] = 1
df.loc[df['value'].isin(zero_vals), 'value'] = 0
df


,date,time,sensor_id,value,activity,status,datetime
0,02/02/2009,7:15:17,M35,1,R1_Bed_to_Toilet,begin,2009-02-02 07:15:17
1,02/02/2009,7:15:21,AD1-A,2.82231,NaN,NaN,2009-02-02 07:15:21
2,02/02/2009,7:15:23,M35,0,NaN,NaN,2009-02-02 07:15:23
3,02/02/2009,7:15:23,M35,1,NaN,NaN,2009-02-02 07:15:23
4,02/02/2009,7:15:28,AD1-A,2.79926,NaN,NaN,2009-02-02 07:15:28
...,...,...,...,...,...,...,...
138625,04/04/2009,7:26:16,M48,0,NaN,NaN,2009-04-04 07:26:16
138626,04/04/2009,7:27:47,M48,1,NaN,NaN,2009-04-04 07:27:47
138627,04/04/2009,7:27:50,M47,1,NaN,NaN,2009-04-04 07:27:50
138628,04/04/2009,7:27:59,M47,0,NaN,NaN,2009-04-04 07:27:59


In [43]:
# 检查begin总数==end总数
def check_begin_end_total(df):
    cnt_begin = 0
    cnt_end = 0
    for index,row in df.iterrows():
        if row.status == 'begin':
            cnt_begin += 1
        elif row.status == 'end':
            cnt_end += 1
    print(cnt_begin)
    print(cnt_end)
check_begin_end_total(df)

514
514


In [44]:
# 检查begin-end pair的activities可以相互抵消
def check_activity_symmetry(df):
    stack = []
    cnt = 0
    for index,row in df.iterrows():
        if not pd.isnull(row.activity):
            if row.status == 'begin':
                stack.append(row.activity)
            elif row.status == 'end':
                stack.pop()

        # 每1000次打印一次栈的状态
        cnt += 1
        if cnt % 1000 == 0:
            print(stack)

    print("The final stack value:", stack)

check_activity_symmetry(df)

['Watch_TV']
['R1_Work', 'R2_Work']
['R1_Work']
['Meal_Preparation']
['Meal_Preparation']
['R1_Sleep', 'R2_Sleep']
['Meal_Preparation']
['R2_Work']
['R2_Work', 'R2_Sleep']
['Meal_Preparation']
['R1_Work']
['R1_Work', 'R2_Work']
['R2_Sleep', 'R1_Sleep']
['Meal_Preparation']
['R1_Work', 'Clean']
['Study', 'R1_Work']
['Study', 'R1_Work']
['R1_Sleep', 'R2_Sleep']
['Meal_Preparation']
['R1_Work']
['Watch_TV']
['Watch_TV', 'R2_Sleep']
['R1_Personal_Hygiene']
['Meal_Preparation']
['Meal_Preparation']
['Meal_Preparation']
[]
['Meal_Preparation']
['R1_Work', 'R2_Sleep']
['R1_Work', 'R1_Sleep', 'R1_Bed_to_Toilet']
['R1_Work', 'R2_Personal_Hygiene']
['Meal_Preparation', 'Watch_TV']
['Meal_Preparation', 'R1_Work']
['Meal_Preparation', 'R1_Work']
['Meal_Preparation', 'R1_Sleep']
['Watch_TV', 'Meal_Preparation']
['Watch_TV', 'Meal_Preparation']
['Watch_TV', 'R1_Work', 'R2_Work']
['R1_Sleep', 'R2_Sleep']
['Meal_Preparation']
['R1_Sleep', 'R2_Work']
['R1_Sleep', 'R1_Work']
['Meal_Preparation']
['Meal_

In [45]:
df

,date,time,sensor_id,value,activity,status,datetime
0,02/02/2009,7:15:17,M35,1,R1_Bed_to_Toilet,begin,2009-02-02 07:15:17
1,02/02/2009,7:15:21,AD1-A,2.82231,NaN,NaN,2009-02-02 07:15:21
2,02/02/2009,7:15:23,M35,0,NaN,NaN,2009-02-02 07:15:23
3,02/02/2009,7:15:23,M35,1,NaN,NaN,2009-02-02 07:15:23
4,02/02/2009,7:15:28,AD1-A,2.79926,NaN,NaN,2009-02-02 07:15:28
...,...,...,...,...,...,...,...
138625,04/04/2009,7:26:16,M48,0,NaN,NaN,2009-04-04 07:26:16
138626,04/04/2009,7:27:47,M48,1,NaN,NaN,2009-04-04 07:27:47
138627,04/04/2009,7:27:50,M47,1,NaN,NaN,2009-04-04 07:27:50
138628,04/04/2009,7:27:59,M47,0,NaN,NaN,2009-04-04 07:27:59


In [46]:
# 对activity填值：begin-end中间填最近的activity，begin-end外填other

# def set_activity(df):
stack = []
cnt = 0
for index,row in df.iterrows():
    if not pd.isnull(row["activity"]):
#         print("yes activity!", row.activity)
#         print(cnt)
        if row.status == 'begin':
            stack.append(row.activity)
        elif row.status == 'end':
            del(stack[stack.index(row.activity)])
    else:
        # 判断有没有被begin-end包围
        if len(stack) == 0:
            df.loc[index, 'activity'] = "other"
        else:
            df.loc[index, 'activity'] = stack[-1]

    cnt += 1
    # return df

df

,date,time,sensor_id,value,activity,status,datetime
0,02/02/2009,7:15:17,M35,1,R1_Bed_to_Toilet,begin,2009-02-02 07:15:17
1,02/02/2009,7:15:21,AD1-A,2.82231,R1_Bed_to_Toilet,NaN,2009-02-02 07:15:21
2,02/02/2009,7:15:23,M35,0,R1_Bed_to_Toilet,NaN,2009-02-02 07:15:23
3,02/02/2009,7:15:23,M35,1,R1_Bed_to_Toilet,NaN,2009-02-02 07:15:23
4,02/02/2009,7:15:28,AD1-A,2.79926,R1_Bed_to_Toilet,NaN,2009-02-02 07:15:28
...,...,...,...,...,...,...,...
138625,04/04/2009,7:26:16,M48,0,R2_Sleep,NaN,2009-04-04 07:26:16
138626,04/04/2009,7:27:47,M48,1,R2_Sleep,NaN,2009-04-04 07:27:47
138627,04/04/2009,7:27:50,M47,1,R2_Sleep,NaN,2009-04-04 07:27:50
138628,04/04/2009,7:27:59,M47,0,R2_Sleep,NaN,2009-04-04 07:27:59


## 1.5 将sensor_id和activities转为数值，用于模型训练

In [47]:
from sklearn.preprocessing import LabelEncoder

# 对sensor进行labelEncoder
sensor_encoder = LabelEncoder()
sensor_encoder.fit(df.sensor_id)

# 对activity进行labelEncoder
activity_encoder = LabelEncoder()
activity_encoder.fit(df.activity.astype(str))

df.sensor_id = sensor_encoder.transform(df.sensor_id)
df.activity = activity_encoder.transform(df.activity.astype(str))

In [48]:
#输出label encoder编码后的值
print(sorted(df.sensor_id.unique()))
print(sorted(df.activity.unique()))

#查看数字对应的原始名字
print(sensor_encoder.inverse_transform(sorted(df.sensor_id.unique())))
print(activity_encoder.inverse_transform(sorted(df.activity.unique())))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
['AD1-A' 'AD1-B' 'AD1-C' 'D03' 'D05' 'D07' 'D08' 'D09' 'D10' 'D12' 'D14'
 'D15' 'I03' 'L04' 'L06' 'L09' 'L10' 'L11' 'L12' 'L13' 'M01' 'M02' 'M03'
 'M04' 'M05' 'M06' 'M07' 'M08' 'M09' 'M10' 'M11' 'M12' 'M13' 'M14' 'M15'
 'M16' 'M17' 'M18' 'M19' 'M20' 'M21' 'M22' 'M23' 'M24' 'M25' 'M26' 'M27'
 'M28' 'M29' 'M30' 'M31' 'M32' 'M33' 'M34' 'M35' 'M36' 'M37' 'M38' 'M39'
 'M40' 'M41' 'M42' 'M43' 'M44' 'M45' 'M46' 'M47' 'M48' 'M49' 'M50' 'M51']
['Clean' 'Meal_Preparation' 'R1_Bed_to_Toilet' 'R1_Personal_Hygiene'
 'R1_Sleep' 'R1_Work' 'R2_Bed_to_Toilet' 'R2_Personal_Hygiene' 'R2_Sleep'
 'R2_Work' 'Study' 'Wash_Bathtub' 'Watch_TV' 'other']


In [49]:
df

,date,time,sensor_id,value,activity,status,datetime
0,02/02/2009,7:15:17,54,1,2,begin,2009-02-02 07:15:17
1,02/02/2009,7:15:21,0,2.82231,2,NaN,2009-02-02 07:15:21
2,02/02/2009,7:15:23,54,0,2,NaN,2009-02-02 07:15:23
3,02/02/2009,7:15:23,54,1,2,NaN,2009-02-02 07:15:23
4,02/02/2009,7:15:28,0,2.79926,2,NaN,2009-02-02 07:15:28
...,...,...,...,...,...,...,...
138625,04/04/2009,7:26:16,67,0,8,NaN,2009-04-04 07:26:16
138626,04/04/2009,7:27:47,67,1,8,NaN,2009-04-04 07:27:47
138627,04/04/2009,7:27:50,66,1,8,NaN,2009-04-04 07:27:50
138628,04/04/2009,7:27:59,66,0,8,NaN,2009-04-04 07:27:59


## 1.6构建mutual matrix

In [50]:
## mutual源代码
# def calculate_mutual_info_matrix(self, mutual_matrix, sensnum):
#     current_activity = self.data.iloc[0].activity1
#     wind = []
#     winds=[]
#     winnum = 0
#
#     for idx, row in self.data.iterrows():
#         if current_activity == row.activity1:
#             wind.append(row.sensor_id)
#         else:
#             sensors = np.unique(wind)
#             for sensor in range(0, sensnum):
#                 for sensor2 in range(0, sensnum):
#                     if sensor in sensors and sensor2 in sensors:
#                         mutual_matrix[sensor][sensor2] += 1
#                         winds.append(wind)
#             winnum += 1
#             current_activity = row.activity1
#             wind = [row.sensor_id]
#
#     return mutual_matrix/winnum
#
# mutual_matrix = self.calculate_mutual_info_matrix(np.zeros((self.sens_num, self.sens_num)),
#                                                                    self.sens_num)

In [51]:
## 构建mutual矩阵-->在数据集activities的前提下，传感器之间的关系
current_activity = df.iloc[0].activity
wind = []
winds=[]
winnum = 0

sens_num = len(df.sensor_id.unique()) # 71
mutual_matrix = np.zeros((sens_num, sens_num))
sensnum = sens_num

for idx, row in df.iterrows():
        # 以当前事件作为判断
        if current_activity == row.activity:
            # 窗口增加：当前行的sensor_id
            # 将与当前事件相关的sensor依次录入wind窗口
            wind.append(row.sensor_id)
        # 首次出现非当前事件-即下个activity
        else:
            # 将窗口里的独特的sensor_id进行排序
            sensors = np.unique(wind)
            # 将当前事件涉及的sensor_id(unique)填入对称的mutual矩阵里
            for sensor in range(0, sensnum):
                for sensor2 in range(0, sensnum):
                    if sensor in sensors and sensor2 in sensors:
                        mutual_matrix[sensor][sensor2] += 1
                        winds.append(wind)
            # 事件数+1
            winnum += 1
            # 重置当前事件
            current_activity = row.activity
            # 重置wind窗口
            wind = [row.sensor_id]

mutual_matrix = mutual_matrix/winnum

In [52]:
mutual_matrix

array([[0.13484487, 0.09546539, 0.04415274, ..., 0.00238663, 0.00835322,
        0.10739857],
       [0.09546539, 0.1372315 , 0.05608592, ..., 0.00238663, 0.00596659,
        0.12291169],
       [0.04415274, 0.05608592, 0.05727924, ..., 0.        , 0.00119332,
        0.05369928],
       ...,
       [0.00238663, 0.00238663, 0.        , ..., 0.06801909, 0.04653938,
        0.00596659],
       [0.00835322, 0.00596659, 0.00119332, ..., 0.04653938, 0.11575179,
        0.00596659],
       [0.10739857, 0.12291169, 0.05369928, ..., 0.00596659, 0.00596659,
        0.15513126]])

## 1.7构建滑动窗口

In [53]:
# from datetime import datetime
# idx = 0
# sliding_window_length = 6
# windows = df.iloc[idx:idx + sliding_window_length]
# windows

In [54]:
# last_activity = windows.iloc[sliding_window_length-1].activity
# 
# start_time = windows.iloc[0].datetime
# last_time = windows.iloc[sliding_window_length - 1].datetime
# last_time = datetime(last_time.year, last_time.month, last_time.day, last_time.hour, last_time.minute,
#                              last_time.second)
# start_time = datetime(start_time.year, start_time.month, start_time.day, start_time.hour, start_time.minute,
#                       start_time.second)
# last_time = last_time.hour / 24  + last_time.minute / 60. + last_time.second / 3600.
# start_time = start_time.hour / 24 + start_time.minute / 60. + start_time.second / 3600.
# print(start_time)
# print(last_time)

In [55]:
# last_sensor_id = windows.iloc[sliding_window_length - 1].sensor_id
# second_last_id = windows.iloc[sliding_window_length - 2].sensor_id
# print(last_sensor_id)
# print(second_last_id)

In [56]:
# sens_num = len(df.sensor_id.unique()) # 71
# freq = np.zeros(sens_num)
# for ind, act in windows.iterrows():
#     freq[int(act.sensor_id)] += 1
# print(freq)

In [ ]:
from datetime import datetime
sliding_window_length = 120 # 30: 92.9329% 94.2965%; 80:96.3623% 97.7589%; 100: 97.412%, 98.581%
windows_vectors, activities_vectors = [], [],
for idx in range(0, df.shape[0], 1):  #[0,139629]
    current_sens = []
    if df.shape[0] <= idx + sliding_window_length:
        break

    sens_num = len(df.sensor_id.unique()) # 71
    freq = np.zeros(sens_num)
    # 取前sliding_window_length行作为windows
    windows = df.iloc[idx:idx + sliding_window_length]
    # 窗口里最后一行的事件
    last_activity = windows.iloc[sliding_window_length-1].activity

    # 将窗口里开始时间和结束时间转为0~1的值
    start_time = windows.iloc[0].datetime
    last_time = windows.iloc[sliding_window_length - 1].datetime
    last_time = datetime(last_time.year, last_time.month, last_time.day, last_time.hour, last_time.minute,
                                 last_time.second)
    start_time = datetime(start_time.year, start_time.month, start_time.day, start_time.hour, start_time.minute,
                          start_time.second)
    last_time = last_time.hour / 24  + last_time.minute / (60. * 24) + last_time.second / (3600. * 24)
    start_time = start_time.hour / 24 + start_time.minute / (60. * 24)  + start_time.second / (3600. * 24)

    last_sensor_id = windows.iloc[sliding_window_length - 1].sensor_id
    # !!!感觉好像没理由用second_last_id, 调参时考虑把它注释掉试试
    second_last_id = windows.iloc[sliding_window_length - 2].sensor_id

    # 记录窗口中每个sensor_id出现的次数
    for ind, act in windows.iterrows():
        freq[int(act.sensor_id)] += 1

    #weighting frequencies using mutual matrix table
    # 按窗口中最后一行的传感器相关性
    for ind, val in enumerate(freq):
        freq[ind] = freq[ind] * mutual_matrix[last_sensor_id][ind]

    # 解决极端情况: last_time = 0.0001 start_time = 0.99999999
    if last_time >= start_time:
        timespan = last_time - start_time
    else:
        timespan = (last_time + 1) - start_time

    if(timespan < 0):
        print("Time span < 0! Error!")


    freq = np.append(freq, (last_time, timespan, second_last_id, last_sensor_id)) # 模型输入维度71+4

    windows_vectors.append(freq)
    activities_vectors.append(last_activity) # last_activity 作为label

In [ ]:
# windows_vectors

In [ ]:
# activities_vectors

In [ ]:
    # def get_segments(self):
    #     windows_vectors, activities_vectors = [], [],
    #
    #     for idx in range(0, self.data.shape[0], 1):
    #         current_sens = []
    #         if self.data.shape[0] <= idx + self.sliding_window_length:
    #             break
    #
    #         freq = np.zeros(self.sens_num)
    #         windows = self.data.iloc[idx:idx + self.sliding_window_length]
    #         last_activity = windows.iloc[self.sliding_window_length - 1].activity1
    #
    #         start_time = windows.iloc[0].datetime
    #         last_time = windows.iloc[self.sliding_window_length - 1].datetime
    #         last_time = datetime(last_time.year, last_time.month, last_time.day, last_time.hour, last_time.minute,
    #                              last_time.second)
    #         start_time = datetime(start_time.year, start_time.month, start_time.day, start_time.hour, start_time.minute,
    #                               start_time.second)
    #         last_time = last_time.hour / 24  + last_time.minute / 60. + last_time.second / 3600.
    #         start_time = start_time.hour / 24 + start_time.minute / 60. + start_time.second / 3600.
    #         last_sensor_id = windows.iloc[self.sliding_window_length - 1].sensor_id
    #         second_last_id = windows.iloc[self.sliding_window_length - 2].sensor_id
    #
    #         for ind, act in windows.iterrows():
    #             freq[int(act.sensor_id)] += 1
    #
    #         #weighting frequencies using mutual matrix table
    #         for ind, val in enumerate(freq):
    #             freq[ind] = freq[ind] * self.mutual_matrix[last_sensor_id][ind]
    #
    #         timespan = last_time - start_time
    #         freq = np.append(freq, (last_time, timespan, second_last_id, last_sensor_id))
    #
    #         windows_vectors.append(freq)
    #         activities_vectors.append(last_activity)
    #
    #     return np.array(windows_vectors), np.array(activities_vectors)

## 1.8划分训练测试集

In [62]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    windows_vectors, activities_vectors, test_size=0.2, random_state=42)

In [63]:
print(np.shape(X_train))
print(np.shape(y_train))
print(np.shape(X_test))
print(np.shape(y_test))

(110824, 75)
(110824,)
(27706, 75)
(27706,)


# 2. 构建模型

In [64]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
import graphviz
from numpy import random
from matplotlib import pyplot as plt
import os

# Below command is used to fix the bug when generating visualized tree in RF.
# only for Windows system.
# os.environ["PATH"] += os.pathsep + "D:/Graphviz/bin/" #记得改文件位置



def DecisionTree_Classifier(x_train, x_valid_test, y_train, y_valid_test):
    """
    First take the training set, and valid or test set as inputs.
    Then classify them binary with DT. Print the information related to classification accuracy.
    Inputs:
        x_train: Preprocessed brain MRI images as inputs to train a model.
        y_train: Label information of x_train as inputs to train a model.
        x_valid_test: Preprocessed brain MRI images to validate or test the classification accuracy of the trained model.
                    The preprocessing of valid or test sets cannot use any information of x_train or y_train.
        y_valid: Label information of valid or test sets to calculate the classification accuracy of the trained model.
    Outputs:
        accu: Accuracy of the model on valid or test set.
        y_pred: Predicted labels on valid or test set.
    """
    # Create RF classifier.
    rf = DecisionTreeClassifier()
    rf.fit(x_train, y_train)  # Fit RF model

    # Plot one of the decision tree
    # text format
    # text_representation = tree.export_text(rf.estimators_[1])
    # print(text_representation)

    # image format
    # ----------------------------------------------------------------
    # dot_data = tree.export_graphviz(rf)
    #
    # graph = graphviz.Source(dot_data)
    # graph.render("DecisionTree")
    # graph.write_pdf("DecisionTree_"+str(seed)+".pdf")
    # ----------------------------------------------------------------

    # Make prediction.
    y_pred = rf.predict(x_valid_test)

    # Print results.
    report = metrics.classification_report(y_valid_test, y_pred)
    # print("Random Forest classification report:\n " + report)
    print("Decision Tree classification report:\n " + report)

    accu = metrics.accuracy_score(y_valid_test, y_pred)
    # print("Random Forest classification accuracy: " + str(accu))
    print("Decision Tree classification accuracy: " + str(accu))

    return accu, y_pred



def RF_Classifier(x_train, x_valid_test, y_train, y_valid_test, n_estimators = 100):
    """
    First take the training set, and valid or test set as inputs.
    Then classify them binary with RF. Print the information related to classification accuracy.
    Inputs:
        x_train: Preprocessed brain MRI images as inputs to train a model.
        y_train: Label information of x_train as inputs to train a model.
        x_valid_test: Preprocessed brain MRI images to validate or test the classification accuracy of the trained model.
                    The preprocessing of valid or test sets cannot use any information of x_train or y_train.
        y_valid: Label information of valid or test sets to calculate the classification accuracy of the trained model.
    Outputs:
        accu: Accuracy of the model on valid or test set.
        y_pred: Predicted labels on valid or test set.
    """
    # Create RF classifier.
    rf = RandomForestClassifier(n_estimators=n_estimators, random_state=0)
    rf.fit(x_train, y_train)  # Fit RF model

    # Plot one of the decision tree
    # text format
    # text_representation = tree.export_text(rf.estimators_[1])
    # print(text_representation)

    # image format
    # ----------------------------------------------------------------
    '''
    seed = random.randint(100)
    dot_data = tree.export_graphviz(rf.estimators_[seed])

    graph = graphviz.Source(dot_data)
    graph.write_pdf("RF_Tree_"+str(seed)+".pdf")
    '''
    # ----------------------------------------------------------------

    # Make prediction.
    y_pred = rf.predict(x_valid_test)

    # Print results.
    report = metrics.classification_report(y_valid_test, y_pred)
    print("Random Forest classification report:\n " + report)

    accu = metrics.accuracy_score(y_valid_test, y_pred)
    print("Random Forest classification accuracy: " + str(accu))

    return accu, y_pred

In [65]:
# Train with training set and predict on test set.
#
score, _ = DecisionTree_Classifier(X_train, X_test, y_train, y_test)

Decision Tree classification report:
               precision    recall  f1-score   support

           0       0.98      0.98      0.98       285
           1       0.99      0.99      0.99      7375
           2       0.97      0.93      0.95       638
           3       0.96      0.97      0.97      1142
           4       0.95      0.95      0.95      2145
           5       0.98      0.98      0.98      5048
           6       0.90      0.91      0.90       364
           7       0.97      0.97      0.97      1597
           8       0.96      0.96      0.96      3588
           9       0.98      0.98      0.98      1935
          10       0.99      0.99      0.99       876
          11       1.00      1.00      1.00        44
          12       0.98      0.99      0.98      2474
          13       0.81      0.79      0.80       195

    accuracy                           0.97     27706
   macro avg       0.96      0.96      0.96     27706
weighted avg       0.97      0.97      0.9

In [66]:
score, _ = RF_Classifier(X_train, X_test, y_train, y_test)

Random Forest classification report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       285
           1       0.99      1.00      1.00      7375
           2       0.98      0.97      0.98       638
           3       0.99      0.99      0.99      1142
           4       0.98      0.94      0.96      2145
           5       0.99      0.99      0.99      5048
           6       0.96      0.96      0.96       364
           7       1.00      0.99      1.00      1597
           8       0.96      0.98      0.97      3588
           9       0.99      0.99      0.99      1935
          10       0.99      0.99      0.99       876
          11       0.96      1.00      0.98        44
          12       1.00      0.99      0.99      2474
          13       0.95      0.85      0.89       195

    accuracy                           0.99     27706
   macro avg       0.98      0.97      0.98     27706
weighted avg       0.99      0.99      0.9